In [4]:
import torch
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import xarray as xr
import seaborn as sb
import torch.nn.functional as F
import pytorch_lightning as pl
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from torchmetrics import F1Score
import wandb
from pytorch_lightning.loggers import WandbLogger



files = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))

/kaggle/input/ocean-eddy-detection/sample_final_submission.csv
/kaggle/input/ocean-eddy-detection/eddies_train.nc
/kaggle/input/ocean-eddy-detection/OSSE_U_V_SLA_SST_test.nc
/kaggle/input/ocean-eddy-detection/OSSE_U_V_SLA_SST_train.nc


In [1]:
!pip install segmentation_models_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=fa0163883c32091ffcff627a7bb7938a73a2dbff27a76850d2000764cdf1993f
  Stored in directory: /root/.cache/pip/wheels/96/3f/5f/13976445f67f3b4e77b054e65f7f4c39016e92e8358fe088db
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=8b4ebfc5a64f96e6273445bdf689565bb572564b30cd228e4a2473ce887e62f9
  Stored in directory: /root/.cache/pip/wheels/4f/89/a3/5cf59e30a8a75c917c313f14da0f6209be2d147e3160b985d6
Successfully built efficientnet-pytorch pretrainedmodels


In [5]:
#9d3a5e657688cac29d67d7fe40d48c8f338079d0
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
# WARNING flatten le résultat en IdxTemps_IdLatitude_IdLongitude
# Keep only needed variable 

eddies_train = xr.open_dataset("/kaggle/input/ocean-eddy-detection/eddies_train.nc")
OSSE_test = xr.open_dataset("/kaggle/input/ocean-eddy-detection/OSSE_U_V_SLA_SST_test.nc")
OSSE_train = xr.open_dataset("/kaggle/input/ocean-eddy-detection/OSSE_U_V_SLA_SST_train.nc")
OSSE_train = OSSE_train.rename({"time_counter":"time"})

selected_var = ['vomecrtyT', 'vozocrtxT','sossheig','votemper']
OSSE_train = OSSE_train.fillna(0.0)
X_full = torch.tensor(OSSE_train.get(selected_var).to_array().to_numpy()) # Features x Nb x Latitude x Longitude
X_full = X_full.permute(1, 0, 2, 3) 

na_value = 3.0
eddies_train = eddies_train.fillna(na_value) # fill coast with 999.
y_full = torch.tensor(eddies_train.to_array().to_numpy()) # Target x Times x Latitude x Longitude 
y_full = y_full.permute(1, 0, 2, 3).long()

In [ ]:
nb_val = X_full.shape[0]
idx_split = int(0.8*nb_val)
X_train = torch.tensor(X_full[:idx_split,: , :, :]).clone().detach()
y_train = torch.tensor(y_full[:idx_split,: , :, :]).clone().detach()
X_val = torch.tensor(X_full[idx_split:, : , :, :]).clone().detach()
y_val = torch.tensor(y_full[idx_split:,: , :, :]).clone().detach()

In [10]:
class OSSE_Dataset(Dataset):
    def __init__(self, OSSE_tensor,eddie_tensor):
        self.OSSE_tensor = OSSE_tensor
        self.eddie_tensor = eddie_tensor
        
    def __len__(self):
        return self.OSSE_tensor.shape[0]

    def __getitem__(self, idx):
        features = self.OSSE_tensor[idx, :, :, :]
        label = self.eddie_tensor[idx, :, :, :]
        
        return features, label

In [11]:
def conv_size_out(hin, win, padding, dilation, kernel_size, stride):
    hout = np.floor(((hin + 2*padding - (dilation * (kernel_size -1 )) -1) / stride ) + 1)
    wout = np.floor(((hin + 2*padding - (dilation * (kernel_size -1 )) -1) / stride ) + 1)
    return hout, wout

def conv_transpose_out(hin, win, padding, dilation, kernel_size, stride):
    hout = np.floor(((hin + 2*padding - (dilation * (kernel_size -1 )) -1) / stride ) + 1)
    wout = np.floor(((hin + 2*padding - (dilation * (kernel_size -1 )) -1) / stride ) + 1)
    return hout, wout

In [13]:
class classic_CNN(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Input 4 x 357 x 717
        self.classes = 4
        self.features = 4 
        self.kernel_size = 3
        self.accuracy = F1Score(task ="multiclass",  num_classes=4)
        # Encoder
        self.encoder = nn.Sequential( 
            nn.Conv2d(self.features, 64, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 32, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 16, 6))
        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16, 32, 6),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 64, 3, stride=2, padding=1, output_padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, self.classes, 3, stride=2, padding=1, output_padding=0))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def loss_fn(self,ypred,ytrue):
        return DiceLoss(mode = 'multiclass', ignore_index = 999)(ypred, ytrue)   # Ypred = NxCxHxW Ytrue NxHxW
    
    def configure_optimizers(self):
        LR = 1e-3
        optimizer = torch.optim.Adam(self.parameters(),lr=LR)
        return optimizer

    def training_step(self,batch,batch_idx):
        x,y = batch[0],batch[1]
        out = self(x)
        out = nn.Softmax(-1)(out)
        loss = self.loss_fn(out,y)
        self.log('train_loss', loss)
        return loss      

    def validation_step(self,batch,batch_idx):
        x,y = batch[0],batch[1]
        label = y
        out = self(x)
        out = nn.Softmax(-1)(out)
        loss = self.loss_fn(out,label)
        logits = torch.argmax(out,dim=1, keepdim=True)
        accu = self.accuracy(logits, label)        
        self.log('valid_loss', loss)
        self.log('train_acc_step', accu)
        return loss, accu

In [14]:
ds_train = OSSE_Dataset(X_train,y_train)
train_dataloader = DataLoader(ds_train, batch_size = 8)

ds_val = OSSE_Dataset(X_val,y_val)
val_dataloader = DataLoader(ds_val, batch_size = 8)

In [15]:
wandb_logger = WandbLogger(project='cnn-lightning', 
                           group='CNN', 
                           job_type='train')

wandb: Currently logged in as: a-osty. Use `wandb login --relogin` to force relogin


In [16]:
model = classic_CNN()
trainer = pl.Trainer(gpus = 1, max_epochs = 10, logger=wandb_logger, enable_progress_bar=True)
trainer.fit(model, train_dataloader, val_dataloader)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:475: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1613: PossibleUserWarning: The number of training batches (29) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [44]:
wandb.finish()